## Final Project

In [19]:
import sys, os, imp
import numpy as np
import matplotlib.pyplot as plt
from scipy.constants import lambda2nu, nu2lambda, c

In [20]:
# lumapiFile = "/opt/lumerical/v221/api/python/lumapi.py"
lumapiFile = "C:\\Program Files\\Lumerical\\v221\\api\\python\\lumapi.py"
lumapi = imp.load_source("lumapi", lumapiFile)
fdtdAPI = lumapi.FDTD(hide = False)

## Parameters

In [21]:
# units
um = 1e-6
nm = 1e-9

In [22]:
# Materials 
sub_material = 'SiO2 (Glass) - Palik'
core_material = 'Si (Silicon) - Palik'

In [23]:
# Waveguide parameters
wvg_width = 500 * nm
wvg_height = 220 * nm
L_box = 2 * um
L_rect = 3 * um
space = 0.2 * um

# Bandwidth operation
option = 3
# C Band simulation: 1520 - 1575 nm (1)
# L Band simulation: 1555 - 1635 nm (2)
# Article Band:      1500 - 1580 nm (3)
if option == 1:
    wvlngth_start = 1.520 * um
    wvlngth_stop = 1.575 * um
elif option == 2:
    wvlngth_start = 1.555 * um
    wvlngth_stop = 1.635 * um
else:
    wvlngth_start = 1.500 * um
    wvlngth_stop = 1.580 * um


filename = "splitter_base" 

## Create base geometry

In [24]:
fdtdAPI.switchtolayout()
fdtdAPI.deleteall()


#INPUT WAVEGUIDE
fdtdAPI.addrect()
fdtdAPI.set('name', 'input wg')
fdtdAPI.set('material', core_material)
fdtdAPI.set('x span', L_rect)
fdtdAPI.set('y span', wvg_width)
fdtdAPI.set('z span', wvg_height)
fdtdAPI.set('y', 0)
fdtdAPI.set('x', -(L_rect/2 + L_box/2))
fdtdAPI.set('z', 0)

#OUTPUT WAVEGUIDES
fdtdAPI.addrect()
fdtdAPI.set('name', 'output wg top')
fdtdAPI.set('material', core_material)
fdtdAPI.set('x span', L_rect)
fdtdAPI.set('y span', wvg_width)
fdtdAPI.set('z span', wvg_height)
fdtdAPI.set('y', wvg_width/2 + space/2)
fdtdAPI.set('x', L_rect/2 + L_box/2)
fdtdAPI.set('z', 0)

fdtdAPI.addrect()
fdtdAPI.set('name', 'output wg bottom')
fdtdAPI.set('material', core_material)
fdtdAPI.set('x span', L_rect)
fdtdAPI.set('y span', wvg_width)
fdtdAPI.set('z span', wvg_height)
fdtdAPI.set('y', -(wvg_width/2 + space/2))
fdtdAPI.set('x', L_rect/2 + L_box/2)
fdtdAPI.set('z', 0)

## Add Solver and monitors

In [25]:
# Solver dimensions
x_FDTD = 0
y_FDTD = 0
z_FDTD = 0
x_span_FDTD = L_box*1.5
y_span_FDTD = L_box*1.5
z_span_FDTD = 0.5*um

mesh_x = 20 * nm
mesh_y = 20 * nm
finer_mesh_size = 2.5 * um

pml_layers = 12 #For all profiles, increasing the number of layers will usually lead to lower reflections.
mesh_accuracy = 2
condition = 'Anti-Symmetric'

In [28]:
fdtdAPI.switchtolayout()
fdtdAPI.select('FDTD')
fdtdAPI.delete()
fdtdAPI.select('mesh')
fdtdAPI.delete()
fdtdAPI.select('opt_fields')
fdtdAPI.delete()
fdtdAPI.select('fom')
fdtdAPI.delete()

# Add Solver FDTD 2D
fdtdAPI.addfdtd()
fdtdAPI.set('dimension', '2D')
fdtdAPI.set("background material", sub_material)
fdtdAPI.set("mesh accuracy", mesh_accuracy)
fdtdAPI.set('force symmetric y mesh', True)
fdtdAPI.set("y min bc", condition )
fdtdAPI.set('pml layers', pml_layers)
fdtdAPI.set("x", x_FDTD)
fdtdAPI.set("x span", x_span_FDTD)
fdtdAPI.set("y", y_FDTD)
fdtdAPI.set("y span", y_span_FDTD)
fdtdAPI.set("z", z_FDTD)

# Add Mesh
fdtdAPI.addmesh() 
fdtdAPI.set("x", x_FDTD)
fdtdAPI.set("x span", finer_mesh_size + 2.0*mesh_x)
fdtdAPI.set("y", y_FDTD)
fdtdAPI.set("y span", finer_mesh_size)
fdtdAPI.set("z", z_FDTD)
fdtdAPI.set("z span", z_span_FDTD)

# Add Monitors
fdtdAPI.addpower()
fdtdAPI.set('name', 'opt_fields')
fdtdAPI.set('monitor type', '2D z-normal')
fdtdAPI.set('x', x_FDTD)
fdtdAPI.set('x span', finer_mesh_size)
fdtdAPI.set('y', y_FDTD)
fdtdAPI.set('y span', finer_mesh_size)

# Add FOM
fdtdAPI.addpower()
fdtdAPI.set('name', 'fom')
fdtdAPI.set('monitor type', '2D x-normal')
fdtdAPI.set('x', finer_mesh_size/2.0)
fdtdAPI.set('y', 0)
fdtdAPI.set('y span', y_span_FDTD)
fdtdAPI.set('z', z_FDTD)
fdtdAPI.set('z span', 1.2*um)